In [651]:
from py2neo import Graph

In [652]:
#graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))
graph = Graph("bolt://54.221.161.251", auth=("neo4j", "launchbaddies"))

In [653]:
type(graph)

py2neo.database.Graph

In [654]:
import pandas as pd

In [655]:
recipes = pd.read_csv('recipes_data.csv')
ingredients = pd.read_csv('ingredients.csv')
meals = pd.read_csv('meals.csv')
cuisines = pd.read_csv('cuisines.csv')

In [656]:
for i in range(len(recipes)):
    row = recipes.iloc[i]
    graph.run("CREATE (:RECIPE {name:'" + row['recipe_name'] + "',rating:" + str(row.rating) + ",difficulty:'" + str(row.difficulty)+"' ,time:'" + str(row.time)+"',num_ingredients:'" + str(row.num_ingredients)+"'})")

In [657]:
ingredients.head()

,_id,ingredient_name,food_type
0,0,Onion,Vegetable
1,1,Rice,Grain
2,2,Miso,Legume
3,3,Tomato,Vegetable
4,4,Goat,Meat


In [658]:
for i in range(len(ingredients)):
    row = ingredients.iloc[i]
    graph.run("CREATE (:INGREDIENT {name:'" + row['ingredient_name'] + "',group: '"+ row.food_type +"'})")

In [659]:
meals

,_id,meal_type,time_of_day
0,0,Breakfast,Morning
1,1,Lunch,Mid-day
2,2,Snack,NaN
3,3,Dinner,Evening
4,4,Dessert,Evening


In [660]:
#for i in range(len(meals)):
    #row = meals.iloc[i]
    #graph.run("CREATE (:MEAL {meal:'" + row['meal_type'] + "',time: '"+ row.time_of_day +"'})")

In [661]:
cuisines

,_id,cuisine_type,origin_country
0,0,Pakistani,Pakistan
1,1,French,France
2,2,Japanese,Japan
3,3,Spanish,Spain


In [662]:
for i in range(len(cuisines)):
    row = cuisines.iloc[i]
    graph.run("CREATE (:COUNTRY {name:'" + row['origin_country'] + "',cuisine: '"+row.cuisine_type+"'})")

In [663]:
for i in range(len(cuisines)):
    row = cuisines.iloc[i]
    print("CREATE (:COUNTRY {name:'" + row['origin_country'] + "'})")

CREATE (:COUNTRY {name:'Pakistan'})
CREATE (:COUNTRY {name:'France'})
CREATE (:COUNTRY {name:'Japan'})
CREATE (:COUNTRY {name:'Spain'})


In [664]:
#for i in range(len(ingredients)):
    #row = ingredients.iloc[i]
    #print("CREATE (:INGREDIENT {name:'" + row['ingredient_name'] + "',group: '"+ row.food_type +"'})")

In [665]:
recipes.head()

,_id,recipe_name,cuisine_id,rating,difficulty,amount_pi,p_ingredient_id,amount_si,s_ingredient_id,num_ingredients,meal_id,salty,spicy,sweet,sour,time
0,0,Goat Biryani,0,9,0,1 kg,4,.5kg,1,18,3,True,True,False,False,180
1,1,French Onion Soup,1,9,8,6 large,0,1.5 cups,15,11,3,True,False,False,False,80
2,2,Miso Soup,2,6,8,3 tbsp,2,2 pieces,8,5,2,True,False,False,False,20
3,3,Falooda,0,0,6,2 cups,14,.25 cups,16,10,4,False,False,True,False,20
4,4,Kare Raisu,2,1,1,.5kg,1,1kg,11,25,1,False,True,True,False,80


In [666]:
for i in range(len(recipes)):
    row = recipes.iloc[i]
    ingredient_id = row['p_ingredient_id']
    graph.run("MATCH (r:RECIPE {name:'" + row['recipe_name'] +"'}), (i:INGREDIENT {name:'" + ingredients.iloc[ingredient_id]['ingredient_name'] + "'}) CREATE (r)-[:USES {amount:'" + row['amount_pi'] + "'}]->(i)")

In [667]:
recipes.head()

,_id,recipe_name,cuisine_id,rating,difficulty,amount_pi,p_ingredient_id,amount_si,s_ingredient_id,num_ingredients,meal_id,salty,spicy,sweet,sour,time
0,0,Goat Biryani,0,9,0,1 kg,4,.5kg,1,18,3,True,True,False,False,180
1,1,French Onion Soup,1,9,8,6 large,0,1.5 cups,15,11,3,True,False,False,False,80
2,2,Miso Soup,2,6,8,3 tbsp,2,2 pieces,8,5,2,True,False,False,False,20
3,3,Falooda,0,0,6,2 cups,14,.25 cups,16,10,4,False,False,True,False,20
4,4,Kare Raisu,2,1,1,.5kg,1,1kg,11,25,1,False,True,True,False,80


In [668]:
for i in range(len(recipes)):
    row = recipes.iloc[i]
    #amt = row['amount_si']
    ingredient_id = row['s_ingredient_id']
    graph.run("MATCH (r:RECIPE {name:'" + row['recipe_name'] +"'}), (i:INGREDIENT {name:'" + ingredients.iloc[ingredient_id]['ingredient_name'] + "'}) CREATE (r)-[:USES {amount:'" + row['amount_si'] + "'}]->(i)")

In [669]:
cuisines

,_id,cuisine_type,origin_country
0,0,Pakistani,Pakistan
1,1,French,France
2,2,Japanese,Japan
3,3,Spanish,Spain


In [670]:
#relationships between meals and their country of origin
for i in range(len(recipes)):
    row = recipes.iloc[i]
    cid = row['cuisine_id']
    graph.run("MATCH (r:RECIPE {name:'" + row['recipe_name'] +"'}), (c:COUNTRY{name:'" + cuisines.iloc[cid]['origin_country'] + "'}) CREATE (r)-[:FROM]->(c)")

In [671]:
print(cuisines.iloc[cid]['origin_country'])

Spain


In [622]:
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [644]:
#veg query
data = list(graph.run("MATCH (one:INGREDIENT)-[:USES]-(ee:RECIPE)-[:USES]-(two:INGREDIENT)WHERE one.group <> 'Meat' AND two.group <> 'Meat' RETURN ee"))

In [646]:
import pprint

In [647]:
def get_unique_values(a_list):

    unique_values = set(a_list)

    for value in unique_values:
        pprint.pprint(value)

print(get_unique_values(data))

Record({'ee': Node('RECIPE', difficulty='8', name='French Onion Soup', num_ingredients='11', rating=9, time='80')})
Record({'ee': Node('RECIPE', difficulty='8', name='Tostada con Tomate, Aceite, y Jamon', num_ingredients='3', rating=2, time='10')})
Record({'ee': Node('RECIPE', difficulty='9', name='Flan ', num_ingredients='5', rating=0, time='80')})
Record({'ee': Node('RECIPE', difficulty='6', name='Falooda', num_ingredients='10', rating=0, time='20')})
Record({'ee': Node('RECIPE', difficulty='8', name='Miso Soup', num_ingredients='5', rating=6, time='20')})
None


In [650]:
get_unique_values(data)

Record({'ee': Node('RECIPE', difficulty='8', name='French Onion Soup', num_ingredients='11', rating=9, time='80')})
Record({'ee': Node('RECIPE', difficulty='8', name='Tostada con Tomate, Aceite, y Jamon', num_ingredients='3', rating=2, time='10')})
Record({'ee': Node('RECIPE', difficulty='9', name='Flan ', num_ingredients='5', rating=0, time='80')})
Record({'ee': Node('RECIPE', difficulty='6', name='Falooda', num_ingredients='10', rating=0, time='20')})
Record({'ee': Node('RECIPE', difficulty='8', name='Miso Soup', num_ingredients='5', rating=6, time='20')})


In [ ]:
for i in get_unique_values(data)
    row = 